In [86]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp fully_connected

## The forward and backward passes

In [87]:
#export
from myfastai.matmul import *
from pathlib import Path
from IPython.core.debugger import set_trace
#from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

def get_data(path = "/home/petr/data/mnist.pkl.gz"):
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [88]:
x_train,y_train,x_valid,y_valid = get_data()

In [89]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [90]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [91]:
x_train.mean(),x_train.std()

(tensor(-7.6999e-06), tensor(1.))

In [92]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [93]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [94]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

### Basic architecture

In [95]:
#export
from torch.nn import init

In [96]:
nh = 50

In [97]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [98]:
def lin(x, w, b): return x@w + b

def relu(x): return x.clamp_min(0.)

In [99]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [100]:
w1.mean(),w1.std()

(tensor(-0.0005), tensor(0.0508))

In [101]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [102]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0296), tensor(0.8144))

In [103]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [104]:
%timeit -n 10 _ = model(x_valid)

6.51 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [105]:
y_train,y_valid = y_train.float(),y_valid.float()

In [106]:
preds = model(x_train)

In [107]:
preds.shape

torch.Size([50000, 1])

In [108]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [109]:
mse(preds, y_train)

tensor(27.0103)

In [110]:
(preds.squeeze(-1) - y_train).pow(2).mean()

tensor(27.0103)

## Gradients and backward pass

In [111]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]
    return inp.g

In [112]:
mse_grad(preds, y_train)

tensor([[-1.9128e-04],
        [ 3.0875e-05],
        [-1.5564e-04],
        ...,
        [-3.0265e-04],
        [-1.7273e-04],
        [-2.9853e-04]])

In [113]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [114]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = torch.einsum("bi,bj->ij", inp, out.g)
    b.g = out.g.sum(0)

In [115]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [116]:
forward_and_backward(x_train, y_train)

In [117]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [118]:
# We cheat a little bit and use PyTorch autograd to check our results.
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [119]:
import torch.nn

def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [120]:
loss = forward(xt2, y_train)

In [121]:
loss.backward()

In [122]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [123]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [124]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", self.inp, self.out.g)
        # Creating a giant outer product, just to sum it, is inefficient!
        # self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [125]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [126]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [127]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [128]:
%time loss = model(x_train, y_train)

CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 50.6 ms


In [129]:
# again, fails on memory
%time model.backward()

CPU times: user 389 ms, sys: 87.4 ms, total: 476 ms
Wall time: 121 ms


### Module forward

In [41]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [42]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [43]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [44]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [45]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [46]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [47]:
%time loss = model(x_train, y_train)

CPU times: user 188 ms, sys: 0 ns, total: 188 ms
Wall time: 47.3 ms


In [48]:
%time model.backward()

CPU times: user 469 ms, sys: 62.7 ms, total: 532 ms
Wall time: 138 ms


### PyTorch way

In [130]:
#export
from torch import nn

In [131]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [132]:
model = Model(m, nh, 1)

In [134]:
%time loss = model(x_train, y_train)

CPU times: user 177 ms, sys: 0 ns, total: 177 ms
Wall time: 55.4 ms


In [135]:
%time loss.backward()

CPU times: user 216 ms, sys: 0 ns, total: 216 ms
Wall time: 56.8 ms
